In [ ]:
import numpy as np
import os.path as op
import pandas as pd

import mne
from mne import setup_volume_source_space
from mne.beamformer import apply_lcmv_cov, make_lcmv
from mne.cov import compute_covariance

from scipy.spatial import KDTree

In [ ]:
#dictionary of raw fif files per patients
raw_paths = {"patient_X": "fif file path"}

#dictionary of coregistartion files per patients
coregs_files = {"patient_X": "trans file path"}

In [ ]:
#dictionaries of epochs files
all_epochs_files = {"patient_X" : "file path"}

high_epochs_files = {"patient_X" : " file path"}

low_epochs_files = {"patient_X" : " file path"}

In [ ]:
#dictionary of MNI coorditnates for each hippocampal cotnact per patient
electrode_coords = {
        "patient_X" : {'EEG00X': [-31.9481,  -17.6862,  -21.6874],
                        'EEG00Y':[-32.1601,  -20.9514, -19.5944]},
        "patient_Y" : {'EEG00X': [27.8007,   -7.4679,  -26.6730],
                        'EEG00Y':[28.5425,  -10.6786,  -24.4242]}
        }

In [ ]:
for subject, path in raw_paths.items():
    mne.set_log_level('ERROR')
    raw_fname = op.abspath( raw_paths[subject])
    raw = mne.io.read_raw_fif(raw_fname, verbose = False)
    info = raw.info

    base_subject = subject.split("_part")[0] 

    data_path = op.abspath("path")
    subjects_dir = op.join(data_path, "freesurfer", "recons")
    coreg_fname = op.abspath(coregs_files[subject])
    coreg = mne.read_trans(coreg_fname, verbose = False)

    #improt 3 sets of epochs
    all_epochs_fname = op.abspath(all_epochs_files[subject])
    all_epochs = mne.read_epochs(all_epochs_fname, verbose = False)

    high_epochs_fname = op.abspath(high_epochs_files[subject])
    high_epochs = mne.read_epochs(high_epochs_fname, verbose = False)

    low_epochs_fname = op.abspath(low_epochs_files[subject])
    low_epochs = mne.read_epochs(low_epochs_fname, verbose = False)

    #filter in the ripple band
    all_epochs = all_epochs.filter(80, 150, verbose = False )
    high_epochs = high_epochs.filter(80, 150, verbose = False)
    low_epochs = low_epochs.filter(80, 150, verbose = False)

    #prepare source space
    surface = op.join(subjects_dir, base_subject , "bem" , "inner_skull.surf")
    src = mne.setup_volume_source_space(
    base_subject, subjects_dir=subjects_dir, pos=10.0, surface=surface, add_interpolator=False, verbose = False
    )  

    conductivity = (0.3,)
    model = mne.make_bem_model(
    subject=base_subject, ico=4, conductivity=conductivity, subjects_dir=subjects_dir, verbose = False
    )
    bem = mne.make_bem_solution(model, verbose = False)

    #compute forward solution 
    fwd = mne.make_forward_solution(
        raw_fname,
        trans=coreg,
        src=src,
        bem= bem,
        meg=True,
        eeg=False,
        mindist=5.0,
        n_jobs=None,
        verbose=False,
        )
    #prepare covariance matrix
    rank = mne.compute_rank(all_epochs, tol=1e-6, tol_kind="relative", verbose = False)
    active_win = (-0.1, 0.1)
    baseline_win = (-0.1, 0.1)
    common_win = (-0.1, 0.1)

    #epochs of no SWRs
    baseline_cov = compute_covariance(
        low_epochs,
        tmin=baseline_win[0],
        tmax=baseline_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )
    #epochs of SWRs
    active_cov = compute_covariance(
        high_epochs,
        tmin=active_win[0],
        tmax=active_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )

    #epochs of both
    cov = compute_covariance(
        all_epochs,
        tmin=common_win[0],
        tmax=common_win[1],
        method="shrunk",
        rank=rank,
        verbose=False,
    )

    noise_cov = mne.make_ad_hoc_cov(all_epochs.info, verbose = False) 

    filters = make_lcmv(
        all_epochs.info, fwd, cov,
        reg=0.05, noise_cov = noise_cov, 
        pick_ori="max-power", 
        weight_norm= None, 
        verbose = False
    )

    #electrdoe coords
    coords_dict = electrode_coords[subject]
    elec_labels = list(coords_dict.keys())
    elec_array = np.array([coords_dict[name] for name in elec_labels]) #in MNI space

    trans_file = op.join(data_path, "coregs", base_subject + '-trans.fif')
    mri_head_t = mne.read_trans(trans_file, verbose=False)

    #get the used coords
    used_vertices = filters['vertices'][0]
    head_coords = src[0]['rr'][used_vertices] #this are in meters

    #covert to MNI
    mni_coords = mne.head_to_mni(head_coords, base_subject, mri_head_t, subjects_dir) #are in mm

    # Use KDTree search to find nearest neighbours
    tree = KDTree(mni_coords)
    dist,idx_in_tree = tree.query(elec_array)

    matched_vertices = used_vertices[idx_in_tree]
    filters_for_electrodes = filters['weights'][idx_in_tree]

    print(f"Mapped {len(matched_vertices)} electrodes for {subject}:")
    for label, idx, d in zip(elec_labels, matched_vertices, dist):
        print(f"  {label} → vertex {idx} (distance: {d:.1f} mm)")
    
    rows = []
    for label, idx, d, coord in zip(elec_labels, matched_vertices, dist, mni_coords[idx_in_tree]):
        rows.append({
            "subject": subject,
            "channel": label,
            "vertex_index": idx,
            "distance_mm": round(d, 2),
            "vertex_coords_mni": coord.tolist()
        })

    s
    results_df = pd.DataFrame(rows)
    results_df.to_csv(f"path_{subject}_voxel_coords.csv", index=False)

